### Importing the necessary modules

In [11]:
import requests
import json
import datetime
import pandas as pd
import smtplib
import schedule
import time

### Defining the urls to fetch the data from

#### These APIs are from:https://apisetu.gov.in/public/marketplace/api/cowin#/Appointment%20Availability%20APIs


In [2]:
urls           = {'state':'https://cdn-api.co-vin.in/api/v2/admin/location/states',
                'district':'https://cdn-api.co-vin.in/api/v2/admin/location/districts/{0}'}
url_to_centres = 'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={0}&date={1}'

### In order to get the center availability get the state and district code using the getID

In [3]:
def getID(state_name, district_name=None, district=True):
    '''API to fetch the state id and the district id'''
    func = lambda x,y,data:[ each.get('{0}_id'.format(x)) for each in data if each.get('{0}_name'.format(x))==y ]
    url = urls.get('state')
    states = json.loads(requests.get(url).content).values()
    result = func('state',state_name,list(states)[0])
    if result and district:
        districts = json.loads(requests.get(urls.get('district').format(result[0])).content).values()
        result_dist = func('district',district_name,list(districts)[0])
        if result_dist:
            return result[0],result_dist[0]
        return result[0],None

In [6]:
# getID('Maharashtra','Akola')

(21, 364)

### The below uses the getID function to retrieve the state/district id and use it for checking the availability

In [4]:
def get_available_centres(state,district, minagelimit=18,date=None):
    '''API to fetch the centres available givent the state and the district '''
    newdatalist = []
    date = date or datetime.date.today().strftime("%d-%m-%Y")
    state_id,district_id = getID(state,district)
    response = requests.get(url_to_centres.format(district_id,date)).content
    centers = json.loads(response)['centers']
    for center in centers:
        session = center.pop('sessions')
        center.update(session[0])
        newdatalist.append(center)
    df = pd.DataFrame(data=newdatalist)
    if not df.empty:
        if any(df['available_capacity'].apply(int).to_list()):
            if any(df['min_age_limit'].apply(lambda x:x==minagelimit)):
                df = df[df['available_capacity']>0]
                df = df[df['min_age_limit']==minagelimit]
                return df
            return df[df['min_age_limit']==minagelimit]
        return df[df['available_capacity']>0]
    return df

In [10]:
# get_available_centres('Maharashtra','Akola','04-04-2021')

,center_id,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,session_id,date,available_capacity,min_age_limit,vaccine,slots


In [9]:
# get_available_centres('Maharashtra','Akola',45,'04-04-21')

,center_id,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,session_id,date,available_capacity,min_age_limit,vaccine,slots
0,615075,Pinjar PHC,,Maharashtra,Akola,Barshitakli,444501,20,77,10:00:00,16:00:00,Free,15b83cbc-c5df-4946-bd26-1de8cd026df8,04-04-2021,63,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
1,658554,Borta Sub Center,,Maharashtra,Akola,Murtizapur,444006,0,0,10:00:00,16:00:00,Free,78f22d57-f4d8-41a9-a288-18268d556bbf,04-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
2,609476,Mauli Maternity Hospital,,Maharashtra,Akola,Akola Corporation,444001,20,76,09:00:00,17:00:00,Paid,fef46c54-5aed-4fb4-9c25-57ca74503c24,04-04-2021,98,45,COVISHIELD,"[09:00AM-11:00AM, 11:00AM-01:00PM, 01:00PM-03:..."
3,615018,Mahan PHC,,Maharashtra,Akola,Barshitakli,444405,20,77,10:00:00,16:00:00,Free,0ff32942-9395-47ca-99bf-eab99b9425f9,04-04-2021,94,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
4,31561,Dahatonda Sub Center,,Maharashtra,Akola,Murtizapur,444107,0,0,10:00:00,16:00:00,Free,ec5e32e9-6867-40d2-96ff-706e99b33f1f,05-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
5,31923,Januna Sub Center,,Maharashtra,Akola,Barshitakli,444401,0,0,10:00:00,16:00:00,Free,9f00dfcd-9811-4736-93a0-33909824e103,06-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
6,32101,Mhaisang Sub Center,,Maharashtra,Akola,Akola,444001,0,0,10:00:00,16:00:00,,3c2df045-11b4-400c-b339-f8d16ee8488c,06-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
7,614190,Dhotra PHC,,Maharashtra,Akola,Murtizapur,444107,20,77,10:00:00,16:00:00,Free,394cd6e3-d41a-4b1e-bb88-0a66366e4ee6,07-04-2021,49,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
8,614236,Jamthi PHC,,Maharashtra,Akola,Murtizapur,444106,20,77,10:00:00,16:00:00,Free,b01c22c3-681d-4844-a5d0-c76d34289571,07-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."
9,615066,Parad PHC,,Maharashtra,Akola,Murtizapur,444107,20,77,10:00:00,16:00:00,Free,5cc638ad-47f0-47b1-966a-3c1916b49424,07-04-2021,50,45,COVISHIELD,"[10:00AM-11:00AM, 11:00AM-12:00PM, 12:00PM-01:..."


### send the notification via email: check this link https://dbader.org/blog/python-send-email

In [8]:
def sendEmail(sender,recipients, subject, data):
    '''Function to send email'''
    sender =sender
    recipient = recipients
    password = "*****" # Your SMTP password for Gmail
    subject = subject
    text = data
    smtp_server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    smtp_server.login(sender, password)
    message = "Subject: {}\n\n{}".format(subject, text)
    smtp_server.sendmail(sender, recipient, message)
    smtp_server.close()

### Next step:
#### schedule a job to get notified for the available centres

In [ ]:
# def job():
#     date="05-04-2021"
#     minagelimit=18
#     result = get_available_centres('Maharashtra','Akola',date=date,minagelimit=minagelimit)
#     print(result)
#     if not result.empty:
#         print("sending email")
#         sendEmail('abc.yz@gmail.com',['abc.xyz@gmail.com'],"Covid vaccine available for age limit {0}".format(minagelimit), result.to_string())
#         schedule.cancel_job(job)
# schedule.every(15).minutes.do(job)

# while True:
#     schedule.run_pending()
#     time.sleep(1)